In [6]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
#parse data 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
#label encoding on categorical data 
import tensorflow as tf
from tensorflow import keras

#FAMA 49CRSP Common Stocks 
df = pd.read_csv('../ee6d2f60cdafb550.csv')
df = df.dropna()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
import sklearn.preprocessing 
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split

#preprocessing here
#sort by date 
df = df.sort_values(by = 'public_date', ascending = True)

#encode integer categories into numbers 
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df.FFI49_desc)
print(integer_encoded)
df.FFI49_desc = integer_encoded
df.divyield_Median = [float(x.strip('%'))/100 for x in df.divyield_Median]
print(df)

#todo: https://www.gsb.stanford.edu/library/articles/databases/links/financial-ratios-suite?fbclid=IwAR0EGNGk9DdxQjEHfdaoUhdY3tNzAWDogYDzuuJi1zT_muL-uJtWQw19Fzk

#get output first 
ewlabels = df.indret_ew
vwlabels = df.indret_vw

df = df.drop(columns = ['indret_ew','indret_vw'])
#3year on year change as a prediction feature, raw pct change 
yoythree = ewlabels.diff(periods = 3)
#3 years rolling percent change, averaged ie. (y1-y2 + (y3-y2)change)/2 
rollavgpct = ewlabels.rolling(3).mean()

#drop first 3 years
df = df.iloc[3:]
ewlabels = ewlabels.iloc[3:]
yoythree = yoythree.iloc[3:]
#yoypctthree = yoypctthree.iloc[3:]
rollavgpct = rollavgpct.iloc[3:]

#add -1 and 1 so the bins will take on bins to be equal and set to max -1 and 1
extrema = pd.Series([-1,1])
ewnlabels = ewlabels.append(extrema)

#make a new output (bucket by percentage?)
enc = KBinsDiscretizer(n_bins=8, encode='ordinal',strategy = 'uniform')
ewnlabels = np.asarray(ewnlabels)
ewnlabels = ewnlabels.reshape((-1,1))
labels_binned = enc.fit_transform(ewnlabels)

labels_binned = labels_binned[:-2]

#1 Split-Timer series data, 0.64 Train, 0.16 dev, 0.2 Test
x_train, x_test, y_train, y_test = train_test_split(df, labels_binned, test_size = 0.2, shuffle = False)
x_tra, x_dev, y_tra, y_dev = train_test_split(x_train, y_train, test_size = 0.2, shuffle = False)

[25 25 25 ... 22 10 47]
       public_date  FFI49_desc  NFIRM  indret_ew  indret_vw  dpr_Median  \
12337     19930831          25      6   0.052904   0.068678       0.460   
12383     19930930          25      6  -0.035253  -0.046242       0.460   
12430     19931031          25      6   0.009319  -0.018318       0.460   
12477     19931130          25      5  -0.022123  -0.014000       0.479   
12524     19931231          25      5   0.096039   0.072342       0.479   
12571     19940131          25      5   0.043779   0.009888       0.479   
12618     19940228          25      6   0.012589   0.039537       0.394   
12665     19940331          25      6  -0.012399  -0.052069       0.394   
12712     19940430          25      6  -0.026779  -0.024747       0.394   
12759     19940531          25      6  -0.001201   0.023906       0.394   
12806     19940630          25      6  -0.001294  -0.037881       0.394   
12853     19940731          25      6   0.023423   0.043118       0.394   
1

In [10]:
model = keras.Sequential([
    keras.layers.Dense(76, activation=tf.nn.sigmoid),
    keras.layers.Dense(60, activation=tf.nn.softmax),
    keras.layers.Dense(40, activation=keras.activations.linear),
    keras.layers.Dense(10, activation=tf.nn.sigmoid),
    keras.layers.Dense(8, activation=tf.nn.softmax)
])

#after searching for learning rates, 0.02 was found to be the optimal 
#after searching for epochs, 50 was found to be the most optimal 

for i in ['sparse_categorical_crossentropy']:
    for j in [tf.train.AdamOptimizer(), tf.train.GradientDescentOptimizer(learning_rate=0.02),
             tf.train.AdagradOptimizer(learning_rate = 0.02)]:
        model.compile(optimizer=j, 
              loss=i,
              metrics=['accuracy'])
        model.fit(np.asarray(x_tra), y_tra, epochs=50)
        test_loss, test_acc = model.evaluate(np.asarray(x_dev), y_dev)
        print("test loss")
        print(test_loss)
        print("test_acc")
        print(test_acc)

Epoch 1/50
5948/5948 [==============================] - 1s 92us/step - loss: 1.1545 - acc: 0.5249
Epoch 2/50
5948/5948 [==============================] - 0s 42us/step - loss: 0.8009 - acc: 0.5642
Epoch 3/50
5948/5948 [==============================] - 0s 44us/step - loss: 0.7720 - acc: 0.5642
Epoch 4/50
5948/5948 [==============================] - 0s 41us/step - loss: 0.7629 - acc: 0.5642
Epoch 5/50
5948/5948 [==============================] - 0s 42us/step - loss: 0.7590 - acc: 0.5642
Epoch 6/50
5948/5948 [==============================] - 0s 40us/step - loss: 0.7573 - acc: 0.5642
Epoch 7/50
5948/5948 [==============================] - 0s 41us/step - loss: 0.7552 - acc: 0.5642
Epoch 8/50
5948/5948 [==============================] - 0s 41us/step - loss: 0.7542 - acc: 0.5642
Epoch 9/50
5948/5948 [==============================] - 0s 41us/step - loss: 0.7543 - acc: 0.5642
Epoch 10/50
5948/5948 [==============================] - 0s 42us/step - loss: 0.7533 - acc: 0.5642
Epoch 11/50
5948/59

5948/5948 [==============================] - 0s 41us/step - loss: 0.7516 - acc: 0.5642
Epoch 33/50
5948/5948 [==============================] - 0s 38us/step - loss: 0.7513 - acc: 0.5642
Epoch 34/50
5948/5948 [==============================] - 0s 39us/step - loss: 0.7515 - acc: 0.5642
Epoch 35/50
5948/5948 [==============================] - 0s 43us/step - loss: 0.7513 - acc: 0.5642
Epoch 36/50
5948/5948 [==============================] - 0s 38us/step - loss: 0.7515 - acc: 0.5642
Epoch 37/50
5948/5948 [==============================] - 0s 39us/step - loss: 0.7517 - acc: 0.5642
Epoch 38/50
5948/5948 [==============================] - 0s 42us/step - loss: 0.7511 - acc: 0.5642
Epoch 39/50
5948/5948 [==============================] - 0s 39us/step - loss: 0.7508 - acc: 0.5642
Epoch 40/50
5948/5948 [==============================] - 0s 40us/step - loss: 0.7513 - acc: 0.5642
Epoch 41/50
5948/5948 [==============================] - 0s 40us/step - loss: 0.7514 - acc: 0.5642
Epoch 42/50
5948/5948 

In [11]:
for i in ['sparse_categorical_crossentropy']:
    for j in [tf.train.AdamOptimizer(), tf.train.GradientDescentOptimizer(learning_rate=0.05),
             tf.train.AdagradOptimizer(learning_rate = 0.05)]:
        model.compile(optimizer=j, 
              loss=i,
              metrics=['accuracy'])
        model.fit(np.asarray(x_tra), y_tra, epochs=100)
        test_loss, test_acc = model.evaluate(np.asarray(x_dev), y_dev)
        print("test loss")
        print(test_loss)
        print("test_acc")
        print(test_acc)

Epoch 1/100
5948/5948 [==============================] - 0s 83us/step - loss: 0.7510 - acc: 0.5642
Epoch 2/100
5948/5948 [==============================] - 0s 37us/step - loss: 0.7510 - acc: 0.5642
Epoch 3/100
5948/5948 [==============================] - 0s 40us/step - loss: 0.7513 - acc: 0.5642
Epoch 4/100
5948/5948 [==============================] - 0s 44us/step - loss: 0.7509 - acc: 0.5642
Epoch 5/100
5948/5948 [==============================] - 0s 39us/step - loss: 0.7512 - acc: 0.5642
Epoch 6/100
5948/5948 [==============================] - 0s 39us/step - loss: 0.7511 - acc: 0.5642
Epoch 7/100
5948/5948 [==============================] - 0s 39us/step - loss: 0.7515 - acc: 0.5642
Epoch 8/100
5948/5948 [==============================] - 0s 37us/step - loss: 0.7513 - acc: 0.5642
Epoch 9/100
5948/5948 [==============================] - 0s 45us/step - loss: 0.7512 - acc: 0.5642
Epoch 10/100
5948/5948 [==============================] - 0s 43us/step - loss: 0.7511 - acc: 0.5642
Epoch 11/

5948/5948 [==============================] - 0s 72us/step - loss: 0.7512 - acc: 0.5642
Epoch 83/100
5948/5948 [==============================] - 0s 67us/step - loss: 0.7512 - acc: 0.5642
Epoch 84/100
5948/5948 [==============================] - 0s 83us/step - loss: 0.7512 - acc: 0.5642
Epoch 85/100
5948/5948 [==============================] - 1s 89us/step - loss: 0.7510 - acc: 0.5642
Epoch 86/100
5948/5948 [==============================] - 0s 56us/step - loss: 0.7512 - acc: 0.5642
Epoch 87/100
5948/5948 [==============================] - 0s 42us/step - loss: 0.7512 - acc: 0.5642
Epoch 88/100
5948/5948 [==============================] - 0s 47us/step - loss: 0.7512 - acc: 0.5642
Epoch 89/100
5948/5948 [==============================] - 0s 45us/step - loss: 0.7509 - acc: 0.5642
Epoch 90/100
5948/5948 [==============================] - 0s 51us/step - loss: 0.7510 - acc: 0.5642
Epoch 91/100
5948/5948 [==============================] - 0s 43us/step - loss: 0.7510 - acc: 0.5642
Epoch 92/100


5948/5948 [==============================] - 0s 40us/step - loss: 0.7522 - acc: 0.5627
Epoch 63/100
5948/5948 [==============================] - 0s 37us/step - loss: 0.7529 - acc: 0.5642
Epoch 64/100
5948/5948 [==============================] - 0s 35us/step - loss: 0.7519 - acc: 0.5642
Epoch 65/100
5948/5948 [==============================] - 0s 37us/step - loss: 0.7532 - acc: 0.5642
Epoch 66/100
5948/5948 [==============================] - 0s 36us/step - loss: 0.7526 - acc: 0.5642
Epoch 67/100
5948/5948 [==============================] - 0s 34us/step - loss: 0.7523 - acc: 0.5642
Epoch 68/100
5948/5948 [==============================] - 0s 42us/step - loss: 0.7518 - acc: 0.5642
Epoch 69/100
5948/5948 [==============================] - 0s 35us/step - loss: 0.7527 - acc: 0.5642
Epoch 70/100
5948/5948 [==============================] - 0s 36us/step - loss: 0.7523 - acc: 0.5639
Epoch 71/100
5948/5948 [==============================] - 0s 36us/step - loss: 0.7529 - acc: 0.5642
Epoch 72/100


Epoch 43/100
5948/5948 [==============================] - 0s 45us/step - loss: 0.7510 - acc: 0.5642
Epoch 44/100
5948/5948 [==============================] - 0s 37us/step - loss: 0.7509 - acc: 0.5642
Epoch 45/100
5948/5948 [==============================] - 0s 37us/step - loss: 0.7509 - acc: 0.5642
Epoch 46/100
5948/5948 [==============================] - 0s 37us/step - loss: 0.7510 - acc: 0.5642
Epoch 47/100
5948/5948 [==============================] - 0s 36us/step - loss: 0.7509 - acc: 0.5642
Epoch 48/100
5948/5948 [==============================] - 0s 37us/step - loss: 0.7509 - acc: 0.5642
Epoch 49/100
5948/5948 [==============================] - 0s 39us/step - loss: 0.7509 - acc: 0.5642
Epoch 50/100
5948/5948 [==============================] - 0s 41us/step - loss: 0.7509 - acc: 0.5642
Epoch 51/100
5948/5948 [==============================] - 0s 42us/step - loss: 0.7506 - acc: 0.5642
Epoch 52/100
5948/5948 [==============================] - 0s 36us/step - loss: 0.7510 - acc: 0.5642
